In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from nltk.stem import PorterStemmer
import nltk
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
class labeling_votes():
    
    def __init__(self, filename):
        self.name = filename
        self.df = pd.read_csv(filename+".csv")
        self.dictionary = {}
        #self.model = SentenceTransformer('bert-base-nli-mean-tokens')
        
    def get_dataframe(self):
        return self.df

    def display(self):
        display(self.df.head())

    def preprocess(self):
        for index,line in enumerate(self.df['text']):
            line = line.replace("\\n","") # Replacing New Lines
            line = re.sub('\S*@\S*\s?','',line)   #Replacing emails
            line = re.sub('[^a-z]+',' ',line) #Replacing alphanumeric characters
            self.df['text'][index] = line
    
    def find_patterns(self):
        master_list = []
        list_counter = -1
        self.df['target'] = ""
        for index,text in enumerate(self.df['text']):
            py_stem = PorterStemmer()
            tokens = [py_stem.stem(word) for word in text.split()]
            if re.search('vote (blue|left|biden|joe|liber)|(joe|biden|harris) will win|(harris|joe|biden) 2020|vote (donald|trump) out|(im|i\'m) vote biden', " ".join(tokens)):
                self.df['target'][index] = "biden"
            elif re.search('vote (red|right|conserv)|(trump|donald) will win|(trump|donald) 2020|vote (biden|joe|harris|harry) out|(im|i\'m) vote trump', " ".join(tokens)):
                self.df['target'][index] = "trump"
            
    def create_example_list(self):
        with open('example.txt') as f:
            data = f.read()
            # reconstructing the data as a dictionary
            self.dictionary = json.loads(data)
            
    def label_data(self):
        embeddings_list = []
        voting_list = []
      #print(self.dictionary.keys())
        for key, value in self.dictionary.items():
            if "biden_criticize" in str(key):
                voting_list.append("trump")
            elif "trump_criticize" in str(key):
                voting_list.append("biden")
            elif "trump_wins" in str(key):
                voting_list.append("trump")
            elif "biden_wins" in str(key):
                voting_list.append("biden")
            embeddings_list.append(self.model.encode(self.dictionary[key]))
            print(self.dictionary.keys())
            print(voting_list)
        for doc_index, text in enumerate(self.df['text']):
            max = 0
            max_index = 0
            if self.df['target'][doc_index] == "":
                print(doc_index)
                if re.search("trump|biden",text):
                    line = re.sub('[^a-z]+',' ',text)
                    test_embeddings = self.model.encode(line)
                    for emb_index, embeddings in enumerate(embeddings_list):
                        value = np.max(cosine_similarity([test_embeddings],embeddings))
                        #print("value: ",value)
                        if value > max:
                            #print(doc_index)
                            #print("index: ",doc_index," ","emb_index: ",emb_index)
                            max = value
                            max_index = emb_index
                    self.df['target'][doc_index] = voting_list[max_index]
    
    def analysis_winner(self):
        self.df['timeline'] = df['timeline'].apply(lambda x: x[:10])

        trump_df = self.df[self.df['target'] == 'trump']
        biden_df = self.df[self.df['target'] == 'biden']

        trump_dictionary = {}
        biden_dictionary = {}

        for value in self.df['timeline'].unique():
            trump_dictionary[value] = len(trump_df[trump_df['timeline'] == value])
            biden_dictionary[value] = len(biden_df[biden_df['timeline'] == value])

        fig = go.Figure(data=[
          go.Bar(name='Trump', x=list(trump_dictionary.keys()), y= list(trump_dictionary.values())),
          go.Bar(name='Biden', x=list(biden_dictionary.keys()), y=list(biden_dictionary.values()))
        ])
        # Change the bar mode
        fig.update_layout(barmode='group', title="15 days span before 2020 Presidential Election")
        fig.show('colab')

        fig = px.histogram(df, x="target", text_auto=True, title="Total votes in 2020 Presidential Election")
        fig.show('colab')
      
    def save_dataframe(self):
        self.df.to_csv("labeled_votes.csv")

In [ ]:
obj = labeling_votes("trimmed_pre_election_punctuation_CNN_20")
obj.preprocess()
obj.find_patterns()
obj.create_example_list()
obj.label_data()
obj.save_dataframe()
obj.analysis_winner()

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sen = model.encode("china wants biden to win badly.")
exp = model.encode("if beijing biden wins the china wins")